In [8]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from matplotlib import gridspec
import pickle
import time
import random
from convnet import *
from remtime import *

In [9]:
## Hyperparameters
NUM_OUTPUT = 10
LEARNING_RATE = 0.01	#learning rate
IMG_WIDTH = 28
IMG_DEPTH = 1
FILTER_SIZE=5
NUM_FILT1 = 8
NUM_FILT2 = 8
BATCH_SIZE = 20
NUM_EPOCHS = 2	 # number of iterations
MU = 0.95

PICKLE_FILE = 'output.pickle'
# PICKLE_FILE = 'trained.pickle'

In [10]:
## Data extracting
m =10000
X = extract_data('t10k-images-idx3-ubyte.gz', m, IMG_WIDTH)
y_dash = extract_labels('t10k-labels-idx1-ubyte.gz', m).reshape(m,1)
X-= int(np.mean(X))
X/= int(np.std(X))
test_data = np.hstack((X,y_dash))


m =50000
X = extract_data('train-images-idx3-ubyte.gz', m, IMG_WIDTH)
y_dash = extract_labels('train-labels-idx1-ubyte.gz', m).reshape(m,1)
print np.mean(X), np.std(X)
X-= int(np.mean(X))
X/= int(np.std(X))
train_data = np.hstack((X,y_dash))

('Extracting', 't10k-images-idx3-ubyte.gz')
('Extracting', 't10k-labels-idx1-ubyte.gz')
('Extracting', 'train-images-idx3-ubyte.gz')
('Extracting', 'train-labels-idx1-ubyte.gz')
33.395157 78.66619


In [11]:
np.random.seed(0)
np.random.shuffle(train_data)

In [12]:
NUM_IMAGES = train_data.shape[0]

## Initializing all the parameters
filt1 = {}
filt2 = {}
bias1 = {}
bias2 = {}
for i in range(0,NUM_FILT1):
	filt1[i] = initialise_param_lecun_normal(FILTER_SIZE, IMG_DEPTH, scale=1.0, distribution='normal')
	bias1[i] = 0
	# v1[i] = 0
for i in range(0,NUM_FILT2):
	filt2[i] = initialise_param_lecun_normal(FILTER_SIZE, NUM_FILT1, scale=1.0, distribution='normal')
	bias2[i] = 0
	# v2[i] = 0
w1 = IMG_WIDTH-FILTER_SIZE+1
w2 = w1-FILTER_SIZE+1
theta3 = initialize_theta(NUM_OUTPUT, (w2/2)*(w2/2)*NUM_FILT2)

bias3 = np.zeros((NUM_OUTPUT,1))
cost = []
acc = []
# pickle_in = open(PICKLE_FILE, 'rb')
# out = pickle.load(pickle_in)

# [filt1, filt2, bias1, bias2, theta3, bias3, cost, acc] = out


print("Learning Rate:"+str(LEARNING_RATE)+", Batch Size:"+str(BATCH_SIZE))

Learning Rate:0.01, Batch Size:20


In [13]:
for epoch in range(0,NUM_EPOCHS):
	np.random.shuffle(train_data)
	batches = [train_data[k:k + BATCH_SIZE] for k in xrange(0, NUM_IMAGES, BATCH_SIZE)]
	x=0
	for batch in batches:
		stime = time.time()
		# LEARNING_RATE =  LEARNING_RATE/(1+epoch/10.0)
		out = momentumGradDescent(batch, LEARNING_RATE, IMG_WIDTH, IMG_DEPTH, MU, filt1, filt2, bias1, bias2, theta3, bias3, cost, acc)
		[filt1, filt2, bias1, bias2, theta3, bias3, cost, acc] = out
		epoch_acc = round(np.sum(acc[epoch*NUM_IMAGES/BATCH_SIZE:])/(x+1),2)
		
		per = float(x+1)/len(batches)*100
		print("Epoch:"+str(round(per,2))+"% Of "+str(epoch+1)+"/"+str(NUM_EPOCHS)+", Cost:"+str(cost[-1])+", B.Acc:"+str(acc[-1]*100)+", E.Acc:"+str(epoch_acc))
		
		ftime = time.time()
		deltime = ftime-stime
		remtime = (len(batches)-x-1)*deltime+deltime*len(batches)*(NUM_EPOCHS-epoch-1)
		printTime(remtime)
		x+=1
		break        

Epoch:0.04% Of 1/2, Cost:[2.30456072], B.Acc:15.0, E.Acc:0.15
########  4Hrs 0Mins 10Secs remaining  ########
Epoch:0.04% Of 2/2, Cost:[2.25804369], B.Acc:40.0, E.Acc:0.0
########  1Hrs 57Mins 49Secs remaining  ########


In [14]:
filt1

{0: array([[[-0.17131862,  0.37879949, -0.05013857, -0.21937741,
          -0.0463314 ],
         [-0.0229543 , -0.33805256,  0.26245713,  0.28832176,
           0.0742174 ],
         [-0.07656346,  0.08179235, -0.15878158,  0.20152868,
           0.10219255],
         [ 0.05497357,  0.24906777, -0.03146207,  0.04680861,
           0.09697316],
         [-0.04069121, -0.02762873,  0.04249851, -0.1925393 ,
          -0.10053033]]]),
 1: array([[[-0.02933309, -0.18582481,  0.02801612,  0.07503462,
          -0.29916212],
         [ 0.38951464,  0.43500253,  0.18296193,  0.09328617,
          -0.17040057],
         [ 0.21707925,  0.03855483, -0.24517895,  0.22986368,
           0.15216646],
         [-0.21165304,  0.0832059 , -0.12874004, -0.06481795,
          -0.24102203],
         [-0.32243273, -0.34496154, -0.08856004,  0.26336143,
           0.36082849]]]),
 2: array([[[ 0.01983577,  0.10727224,  0.3066765 ,  0.03141089,
          -0.02572307],
         [ 0.28612071,  0.09426569,  0.